In [35]:
from scripts.preprocess_ter import *
from models.GIN import *

In [5]:
# dir_dataset = "../dataset/" + task + "/" + dataset + "/"
radius = 1
device = torch.device("cpu")
atom_dict = defaultdict(lambda: len(atom_dict))
bond_dict = defaultdict(lambda: len(bond_dict))
fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
edge_dict = defaultdict(lambda: len(edge_dict))

with open("../data/raw_data/train_merged.txt", "r") as f:
    smiles_property = f.readline().strip().split()
    data_original = f.read().strip().split("\n")

"""Exclude the data contains '.' in its smiles."""
data_original = [data for data in data_original if "." not in data.split()[0]]

dataset = []

for data in data_original:
    smiles, property = data.strip().split()

    """Create each data with the above defined functions."""
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    atoms = create_atoms(mol, atom_dict)
    molecular_size = len(atoms)
    i_jbond_dict = create_ijbonddict(mol, bond_dict)
    fingerprints = extract_fingerprints(
        radius, atoms, i_jbond_dict, fingerprint_dict, edge_dict
    )
    adjacency = Chem.GetAdjacencyMatrix(mol)

    """Transform the above each data of numpy
    to pytorch tensor on a device (i.e., CPU or GPU).
    """
    fingerprints = torch.LongTensor(fingerprints).to(device)
    adjacency = torch.FloatTensor(adjacency).to(device)

    property = torch.FloatTensor([[float(property)]]).to(device)

    dataset.append((fingerprints, adjacency, molecular_size, property))


dataset_train = dataset
dataset_train, dataset_dev = split_dataset(dataset_train, 0.9)
# dataset_test = create_dataset("data_test.txt")

# N_fingerprints = len(fingerprint_dict)

# dataset_train, dataset_dev, dataset_test, N_fingerprints

In [21]:
len(dataset)

3990

In [32]:
dataset[3989]

(tensor([ 67,  68, 107,   3,  12,  11,   5,   8,  57,  26,  11,   5,   5, 108,
         109,   5,  73,  74,  31,  53,  31,  19,   5,  45,  46,  14,  60,  35,
          24,   1, 107,   3,  13,  34,   0,   0,  20,  20,  20,  21,  22,  21,
          22,  22,  22,  22,  22,  20,  20,  20,  20,  20,  20,  20,  20,  21,
          20,  20,  20,  20,  20,  20,  20,  20]),
 tensor([[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 64,
 tensor([[6.0070]]))

In [52]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

batch_size = 32

i = 0
for i in range(3900):
    fingerprints_tensor = torch.Tensor(dataset[i][0])
    adjacency_matrix_tensor = torch.Tensor(dataset[i][1])
    molecular_size_tensor = torch.Tensor(dataset[i][2])
    target_tensor = dataset[i][3]

    sample_data = Data(
        x=fingerprints_tensor,
        edge_index=adjacency_matrix_tensor,
        molecular_size=molecular_size_tensor,
        y=target_tensor,
    )
    dataset.append(sample_data)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [56]:
for batch in train_dataloader:
    print(batch)

TypeError: expected Tensor as element 1 in argument 0, but got tuple

In [53]:
import argparse
import json
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torchsummary import summary
import pandas as pd

In [54]:
hidden_dim = 256
input_dim = 1
lr = 0.001
epochs = 100
model = GIN(hidden_dim, input_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()
mae = nn.L1Loss()

for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_mae = 0  # Nouvelle variable pour stocker la somme des MAE
    for batch in train_dataloader:
        optimizer.zero_grad()
        x, edge_index, batch_data = batch.x, batch.edge_index, batch.batch
        output = model(x, edge_index, batch_data)
        loss = criterion(output, batch.y.view(-1, 1))
        mae_value = mae(output, batch.y.view(-1, 1))  # Calcul de la MAE
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_mae += mae_value.item()

    average_loss = total_loss / len(train_dataloader)
    average_mae = total_mae / len(train_dataloader)

    print(f"Epoch {epoch + 1}, Loss: {average_loss}, MAE: {average_mae}")

AttributeError: 'tuple' object has no attribute 'stores'

In [ ]:
import torch
from torch_geometric.data import Data


class MolecularGraphData(Data):
    def __init__(self, fingerprints, adjacency_matrix, molecular_size, target):
        super(MolecularGraphData, self).__init__()
        self.x = fingerprints  # Node features (fingerprints)
        self.edge_index = adjacency_matrix  # Edge connectivity
        self.molecular_size = molecular_size  # Size of the molecular graph
        self.y = target  # Target property

In [76]:
import json

def onehot_encode(x, features:list):
    """
    Maps input elements x not in features to the last element
    """
    if x not in features: x = features[-1]
    binary_encoding = [int(bool_val) for bool_val in list(
       map(lambda s: x == s, features))]
    return binary_encoding
  
class onehot_encodings:
  ''' encoding class for one hot features'''
  def __init__(self, atom_info_func, features):
    self.atom_info_func = atom_info_func
    self.features = features
  
  #@property
  def onehot_encodings(self, atom):
    return onehot_encode(self.atom_info_func(atom), self.features)
  
  def __len__(self): return len(self.features)

def json_to_list(json_file):
  with open(json_file, 'r') as f:
    input_data = list(json.load(f).values())
  print(f"Loaded json from: {json_file}")
  return input_data

def list_to_json(json_file, input_list):
  with open(json_file, 'w') as f:
    json.dump(input_list, f)
  print(f"Output json saved to: {json_file}")
  
from rdkit import Chem
from pathlib import Path
import torch


class FeaturesArgs:
    # encodings information
    available_atoms = [
        "C",
        "N",
        "O",
        "S",
        "F",
        "Si",
        "P",
        "Cl",
        "Br",
        "Mg",
        "Na",
        "Ca",
        "Fe",
        "As",
        "Al",
        "I",
        "B",
        "V",
        "K",
        "Tl",
        "Yb",
        "Sb",
        "Sn",
        "Ag",
        "Pd",
        "Co",
        "Se",
        "Ti",
        "Zn",
        "Li",
        "Ge",
        "Cu",
        "Au",
        "Ni",
        "Cd",
        "In",
        "Mn",
        "Zr",
        "Cr",
        "Pt",
        "Hg",
        "Pb",
        "Unknown",
    ]
    chirality = [
        "CHI_UNSPECIFIED",
        "CHI_TETRAHEDRAL_CW",
        "CHI_TETRAHEDRAL_CCW",
        "CHI_OTHER",
    ]
    num_hydrogens = [0, 1, 2, 3, 4, "MoreThanFour"]
    n_heavy_atoms = num_hydrogens
    formal_charges = [-3, -2, -1, 0, 1, 2, 3, "Extreme"]
    hybridisation_type = ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"]
    # Atoms
    # atom encodings
    atom_encoding_lambdas = {
        "available_atoms": onehot_encodings(
            lambda atom: str(atom.GetSymbol()), available_atoms
        ),
        "chirality_type_enc": onehot_encodings(
            lambda atom: str(atom.GetChiralTag()), chirality
        ),
        "hydrogens_implicit": onehot_encodings(
            lambda atom: int(atom.GetTotalNumHs()), num_hydrogens
        ),
        "n_heavy_atoms": onehot_encodings(
            lambda atom: int(atom.GetDegree()), n_heavy_atoms
        ),
        "formal_charge": onehot_encodings(
            lambda atom: int(atom.GetFormalCharge()), formal_charges
        ),
        "hybridisation_type": onehot_encodings(
            lambda atom: str(atom.GetHybridization()), hybridisation_type
        ),
    }
    # atom info
    atom_info_lambdas = {
        "is_in_a_ring_enc": lambda atom: [int(atom.IsInRing())],
        "is_aromatic_enc": lambda atom: [int(atom.GetIsAromatic())],
        "atomic_mass_scaled": lambda atom: [float((atom.GetMass() - 10.812) / 116.092)],
        "vdw_radius_scaled": lambda atom: [
            float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5) / 0.6)
        ],
        "covalent_radius_scaled": lambda atom: [
            float(
                (Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)
                / 0.76
            )
        ],
    }
    # compute node feature length
    n_node_features = sum(map(len, atom_encoding_lambdas.values()))
    n_node_features += len(atom_info_lambdas)

    # Bonds encoding info
    bond_types = [
        Chem.rdchem.BondType.SINGLE,
        Chem.rdchem.BondType.DOUBLE,
        Chem.rdchem.BondType.TRIPLE,
        Chem.rdchem.BondType.AROMATIC,
    ]
    stereo_types = ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"]
    # bond encodings
    bond_encoding_lambdas = {
        "bond_types": onehot_encodings(lambda bond: bond.GetBondType(), bond_types),
        "stereo_types": onehot_encodings(
            lambda bond: str(bond.GetStereo()), stereo_types
        ),
    }
    # bond quantity
    bond_info_lambas = {
        "bond_is_conj_enc": lambda bond: [int(bond.GetIsConjugated())],
        "bond_is_in_ring_enc": lambda bond: [int(bond.IsInRing())],
    }
    n_edge_features = sum(map(len, bond_encoding_lambdas.values()))
    n_edge_features += len(bond_info_lambas)
    #
    n_features = n_edge_features + n_node_features
    # Molecule
    # lambda mol: GraphDescriptors.BalabanJ(mol)


class ModelArgs:
    available_models = ["GIN"]
    model = "GIN"


class TrainArgs:
    batch_size = 2**5
    lr = 5e-3
    weight_decay = 5e-4
    epochs = 20
    name = "default-GIN"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_save_pt = Path(f"./model/{name}.pth")


class InferArgs:
    batch_size = 1
    device = "cuda" if torch.cuda.is_available() else "cpu"
    name = "default-GIN"
    output_path = Path("./output")
    model_save_pt = Path(f"./model/{name}.pth")


class DataArgs:
    data_source = Path("data/kinase_JAK.csv")
    num_workers = 4
    device = TrainArgs.device

In [77]:
import argparse
import json
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from models.GIN import GIN
from scripts.preprocess import MolDataset
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data
from rdkit import Chem
import networkx as nx

def load_params(config_path):
    with open(config_path, "r") as config_file:
        params = json.load(config_file)
    return params

In [78]:
merged_data = pd.read_csv(
    "/Users/emanieluu/Documents/ENSAE/3A/Advanced Machine Learning/aml_project/data/raw_data/train_merged_data.csv",
    index_col=0
)
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)
train_dataset = MolDataset(train_data)
test_dataset = MolDataset(test_data)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [81]:
# RDkit
from rdkit import Chem
from rdkit.Chem import GraphDescriptors
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

import numpy as np, pandas as pd
import torch
from torch_geometric.data import Data

# from .util import onehot_encodings

def get_atom_features(
    atom,
    available_atoms: list = FeaturesArgs.available_atoms,
    atom_encode_lambdas: dict = FeaturesArgs.atom_encoding_lambdas,
    atom_info_lambdas: dict = FeaturesArgs.atom_info_lambdas,
    debug=False,
):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """
    if "hydrogens_implicit" in atom_encode_lambdas:
        available_atoms = ["H"] + available_atoms
    atom_feature_vector = []
    # compute atom features
    for name, atom_encoding in atom_encode_lambdas.items():
        encoding = atom_encoding.onehot_encodings(atom)
        atom_feature_vector += encoding
        if debug:
            print(f"atom encoding length ({name}): {len(encoding)}")
        # boolean features
    for name, info_func in atom_info_lambdas.items():
        atom_feature_vector += info_func(atom)
        if debug:
            print(f"atom info ({name}): {info_func(atom)}")
    if debug:
        print(f"full atom feature:{len(atom_feature_vector)}")
    return torch.Tensor(atom_feature_vector)


def get_bond_features(
    bond,
    bond_encoding_lambdas: dict = FeaturesArgs.bond_encoding_lambdas,
    bond_info_lambdas: dict = FeaturesArgs.bond_info_lambas,
    debug=False,
):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """
    bond_feature_vector = []
    # compute bond features
    for name, bond_encoding in bond_encoding_lambdas.items():
        encoding = bond_encoding.onehot_encodings(bond)
        bond_feature_vector += encoding
        if debug:
            print(f"bond encoding length ({name}): {len(bond_feature_vector)}")
        # boolean features
    for name, info_func in bond_info_lambdas.items():
        bond_feature_vector += info_func(bond)
        if debug:
            print(f"bond info ({name}): {info_func(bond)}")
        return torch.Tensor(bond_feature_vector)


def smile_to_data(smiles, y_val):
    """smile to pyg Data components"""
    # convert SMILES to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)
    # get feature dimensions
    n_nodes = mol.GetNumAtoms()
    n_edges = 2 * mol.GetNumBonds()

    # construct node feature matrix X of shape (n_nodes, n_node_features)
    for n, atom in enumerate(mol.GetAtoms()):
        atom_features = get_atom_features(atom)
        if n == 0:
            X = torch.zeros((n_nodes, len(atom_features)), dtype=torch.float)
        X[atom.GetIdx(), :] = atom_features

    # construct edge index array E of shape (2, n_edges)
    E_ij = torch.stack(
        list(
            map(
                lambda arr: torch.Tensor(arr).to(torch.long),
                np.nonzero(GetAdjacencyMatrix(mol)),
            )
        )
    )
    # construct edge feature array EF of shape (n_edges, n_edge_features)
    EF = torch.stack(
        [
            get_bond_features(mol.GetBondBetweenAtoms(i.item(), j.item()))
            for i, j in zip(E_ij[0], E_ij[1])
        ]
    )
    # construct label tensor
    y_tensor = torch.tensor(np.array([y_val]), dtype=torch.float)
    return X, E_ij, EF, y_tensor


def graph_datalist_from_smiles_and_labels(x_smiles, y):
    """
    Inputs:
      x_smiles [list]: SMILES strings
      y [list]: numerial labels for the SMILES strings
    Outputs:
      data_list [list]: torch_geometric.data.Data objects which represent labeled molecular graphs that can readily be used for machine learning

    """
    data_list = []
    for smiles, y_val in zip(x_smiles, y):
        X, E, EF, y_tensor = smile_to_data(smiles, y_val)
        # construct Pytorch Geometric data object list
        data_list.append(Data(x=X, edge_index=E, edge_attr=EF, y=y_tensor))
    return data_list


def Molecule_pKa_dataloader(
    df,
    batch_size,
    num_workers: int = 1,
    shuffle=False,
    device="cpu",
    processed_dataset_path="./data",
):
    smile_col, label_col = "SMILES", "measurement_value"
    x_smiles, y = df[smile_col].to_numpy(), df[label_col].to_numpy()

    # create list of molecular graph objects from list of SMILES x_smiles and list of labels y
    data_list = graph_datalist_from_smiles_and_labels(x_smiles, y)
    dataset = Molecule_pKa(processed_dataset_path, data_list)
    # create dataloader for training
    dataloader = DataLoader(
        dataset=data_list,
        batch_size=batch_size,
        num_workers=num_workers,
        collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)),
        shuffle=shuffle,
    )
    return dataloader

In [82]:
def smile_to_data(smiles, y_val):
    """smile to pyg Data components"""
    # convert SMILES to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)
    # get feature dimensions
    n_nodes = mol.GetNumAtoms()
    n_edges = 2 * mol.GetNumBonds()

    # construct node feature matrix X of shape (n_nodes, n_node_features)
    for n, atom in enumerate(mol.GetAtoms()):
        atom_features = get_atom_features(atom)
        if n == 0:
            X = torch.zeros((n_nodes, len(atom_features)), dtype=torch.float)
        X[atom.GetIdx(), :] = atom_features

    # construct edge index array E of shape (2, n_edges)
    E_ij = torch.stack(
        list(
            map(
                lambda arr: torch.Tensor(arr).to(torch.long),
                np.nonzero(GetAdjacencyMatrix(mol)),
            )
        )
    )
    # construct edge feature array EF of shape (n_edges, n_edge_features)
    EF = torch.stack(
        [
            get_bond_features(mol.GetBondBetweenAtoms(i.item(), j.item()))
            for i, j in zip(E_ij[0], E_ij[1])
        ]
    )
    # construct label tensor
    y_tensor = torch.tensor(np.array([y_val]), dtype=torch.float)
    return X, E_ij, EF, y_tensor

In [89]:
X = train_data.loc[0].smiles
y = train_data.loc[0].y

In [91]:
X, E_ij, EF, y_tensor = smile_to_data(X, y)

In [95]:
datalist = graph_datalist_from_smiles_and_labels(train_data["smiles"], train_data["y"])

Data(x=[28, 79], edge_index=[2, 62], edge_attr=[62, 9], y=[1])
